# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [3]:
# The os module is a standard Python library that provides utilities for interacting with the operating system, 
# including environment variable management and file system access.
# In this specific code, it is used to fetch environment variables (such as API keys) from the system or to access file paths.
import os

# The requests library is used to make HTTP requests to retrieve the contents of a web page. 
# In this project, it will be used to fetch the HTML of the page that needs to be summarized.
# This library simplifies making HTTP requests, such as GET and POST, handling network errors, 
# and providing access to response content in different formats (text, JSON, etc.).
# This step is essential to retrieve and prepare text data before summarization using GPT-4.
import requests

# JSON is a common format for data exchange with web services or APIs. Here, it will be used to parse/send structured data.
import json # This module is used to handle JSON (JavaScript Object Notation) data in Python.

# Python's typing module allows you to specify types for variables & function arguments, improving code readability and making it easier to catch errors.
# Certain functions in this project expect lists as input, providing more clarity on data structures passed between functions.
from typing import List

# The load_dotenv function is used to load environment variables from a .env file into the Python environment. 
# .env files contain environment-specific variables and are loaded by dotenv to prevent hard-coding sensitive information in the script.
# The OpenAI API key, which will be required to interact with GPT-4, is stored in this .env file.
from dotenv import load_dotenv

# BeautifulSoup is part of the bs4 library and is used to parse and extract data from HTML and XML documents. 
# After fetching the web page content using requests, BeautifulSoup helps extract clean, readable text from the raw HTML, 
# which can then be summarized by the GPT-4 model.
from bs4 import BeautifulSoup

# These functions are used to  for rendering content in a Jupyter notebook.
# display allows you to render different types of objects in an IPython (Jupyter) environment. 
# Markdown renders text formatted using markdown syntax (like headings, bold, italics, etc.)
# update_display: Allows updating displayed output without re-executing a cell.
# These functions will be used to show the summarized brochures, and other output generated by the LLM, in a readable format in the notebook.
from IPython.display import Markdown, display, update_display

# This imports the OpenAI library, which allows interaction with OpenAI’s models (including GPT-4). 
# The script will use this library to send text data to the GPT-4 model and retrieve the summarized version.
# The OpenAI Python package provides an interface for making requests to OpenAI’s API. 
# This is where API keys and requests are configured to interact with GPT-4, specifying parameters like the model to use, prompt, and temperature.
from openai import OpenAI

## Purpose of the Code:
#### **Setting up API Key Security:** 
Using dotenv to securely load and set environment variables ensures that sensitive information like API keys is not exposed in the codebase.
#### **Model Flexibility:** 
By defining the model as a constant (MODEL = 'gpt-4o-mini'), the code is flexible and easy to modify if you need to switch between different GPT models.
#### **OpenAI Initialization:** 
Initializing the OpenAI client (openai = OpenAI()) sets up the connection with the OpenAI API, enabling further interactions like sending prompts and receiving responses from GPT-4.

In [ ]:
# Load the environment variables defined in the .env file into the current Python environment.
load_dotenv()

# This line ensures that the OpenAI API key is stored in the os.environ dictionary. 
# This dictionary holds all environment variables, which can be accessed by the script at runtime.
# If the key is not found in the environment, it defaults to 'your-key-if-not-using-env'.
# os.environ['OPENAI_API_KEY']: This explicitly sets the environment variable OPENAI_API_KEY to be used by the OpenAI client.
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

MODEL = 'gpt-4o-mini'

# This initializes an instance of the OpenAI class, which will be used to interact with the OpenAI API (in this case, GPT-4).
# The OpenAI() class is part of the OpenAI Python package, which provides an interface for communicating with OpenAI’s API. 
# Once initialized, it enables you to call different models (like GPT-4) and send prompts for completion, summarization, and other NLP tasks.
openai = OpenAI()

## **href attribute**
The href attribute is used in HTML to specify the hyperlink reference for an element. It defines the destination URL that the user will be taken to when they click on the element. Here are some common elements that use the href attribute:

* <a> (anchor) element: This is the most common element that uses the href attribute. It creates a clickable link that, when clicked, takes the user to the specified URL.
* <link> link element: This element is used to link to external resources, such as stylesheets or scripts. The href attribute specifies the location of the resource.
* <img> img element: The href attribute is used with the <img> element to specify the URL of the image file.
  ### Example:
  < a href="https://www.example.com" > Click here to go to Example.com</a>

In [7]:
# A class that encapsulates the core details of a webpage such as its URL, title, body text, and links.

class Website:
    url:str # URL of the webpage (as a string)
    title:str # Title of the webpage (as a string).
    body:str #  Raw HTML content of the webpage (as a string).
    links:List[str] # A list of all the links (<a> tags) found on the webpage.

    # This constructor method is called when a Website object is created.It takes a single argument: the URL of the webpage.
    def __init__(self, url):
        self.url = url # Stores the URL passed into the class instance
       
        response = requests.get(url) # Make an HTTP GET request to the specified URL using the requests library and fetches the webpage content.
        
        self.body = response.content # Stores the raw HTML content of the webpage in the body attribute.
       
        soup = BeautifulSoup(self.body, 'html.parser') # Passes the raw HTML content to BeautifulSoup for parsing, allowing easy access to different parts of the HTML document.
       
        # Extracts the title of the webpage (from the <title> tag) if it exists. If not, it defaults to 'No title found'.
        self.title = soup.title.string if soup.title else 'No title found'

        #  Extracting and Cleaning the Body Text
        if soup.body: # Ensures that the webpage has a body tag (<body>). If it exists, the code proceeds to clean the content and extract the text.
            # Loops through and removes irrelevant HTML elements (<script>, <style>, <img>, <input>) that don’t contain meaningful textual content for summarization.
            for irrelevant in soup.body(['script','style','img','input']):
                irrelevant.decompose() # decompose() ensures these elements are completely removed from the document.
            
            self.text = soup.body.get_text(separator='\n', strip=True) # Extract the remaining clean text from the <body> tag.
            # separator='\n': Ensures that different sections of text are separated by new lines for better readability.
            # strip=True: Removes extra leading and trailing whitespace from the text.
        
        else: # If the <body> tag doesn’t exist, it sets self.text to an empty string.
            self.text = ""

        # Extracting Links:
        # soup.find_all('a'): Finds all anchor (<a>) tags in the HTML document. Anchor tags typically contain hyperlinks, and find_all() retrieves all occurrences.
        links = [link.get('href') for link in soup.find_all('a')] # Extracts the href attribute (the URL) of each <a> tag and stores them in a list (links).

        # Cleaning Links:
        # Filters out any None or empty values from the links list (e.g., if some <a> tags don’t contain an href attribute).
        self.links = [link for link in links if link]

    # Method to Return Webpage Contents:
    def get_contents(self):
        # This uses Python's f-string formatting to return the title and text in a readable format, separating them with new lines.
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        # The clean and structured text ensures better performance when interacting with the LLM.


## ed = Website("https://edwarddonner.com")
**What happens under the hood:**
* The __init__ method of the Website class is called with the URL "https://edwarddonner.com".
* The code then makes an HTTP GET request to this URL using requests.get(url).
* The HTML content of the webpage is fetched and parsed with BeautifulSoup.
* The webpage's title, clean body text, and all links are extracted and stored as attributes of the Website object (ed).
* Now, ed holds all the extracted content for this webpage, including its title, body text, and hyperlinks.

In [8]:
ed = Website("https://edwarddonner.com") # Create an instance of the Website class, named ed, using the URL "https://edwarddonner.com".

# The title of the webpage is printed under "Webpage Title".
# The main text content (after removing irrelevant elements like scripts, styles, images, etc.) is printed under "Webpage Contents".
print(ed.get_contents()) # Display the returned string from get_contents() in the console or Jupyter notebook.

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

### First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

**Constructing a System Prompt for Link Filtering**
* This block of code constructs a system prompt for an LLM, instructing it to process a list of links found on a webpage and decide which links are relevant to include in a company brochure. The system will respond in JSON format with links categorized by type (e.g., About page, Careers page).


**Instructions to the LLM:**
The LLM is told that it will be given a list of links found on a webpage. The task is to determine which links are most relevant for a company brochure. Specifically, links that point to:
 * About page: Pages that describe the company.
 * Company page: General pages about the business.
 * Careers/Jobs pages: Pages related to employment opportunities.
   
The prompt guides the LLM to focus on links that would be useful for building a company brochure, such as About pages and Careers pages.

**Why JSON?:**

_**JSON**_ (JavaScript Object Notation) is a lightweight data format that is easily readable and used for data interchange. By instructing the LLM to respond in JSON format, the output can be parsed programmatically and used in further applications (e.g., filtering the most relevant links).

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage.\
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example: "
# Providing a JSON Example:
# The snippet below provides an example of how the LLM should structure its output. 
# The response should be a dictionary with a links key, and the value should be a list of dictionaries.
# "links": This is a key in the JSON object, and it holds a list of link objects.
# "type": A description of the link type (e.g., "about page", "careers page").
# "url": The full URL corresponding to that type of page.
link_system_prompt += """
{
    "links" : [ 
    {"type" : "about page" , "url" : "https://full.url/goes/here/about" },
    {"type" : "careers page" : "url" : "https://another.full.url/careers"}
    ]
}
"""

**Creating a User Prompt for Link Filtering**

This code defines a function, _**get_links_user_prompt**_, that generates a user-specific prompt for an LLM to decide which links on a company's website are relevant for a brochure. This function complements the previously defined system prompt and combines the webpage’s data (links) with the instruction to the LLM.

In [10]:
# The function dynamically create a user prompt that provides the list of links craped from the given webpage and asks the LLM to decide which ones are relevant for inclusion in a company brochure.
def get_links_user_prompt(website):
    # a. Initial Prompt Setup:
    # This creates the initial part of the user prompt by including the URL of the website that was passed to the function (website.url).
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    # b. Instruction to Filter Relevant Links:
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links. \n"
    # c. Listing Links (Some Might Be Relative Links):
    # This informs the LLM that some links on the webpage might be relative links (i.e., they are not full URLs and need to be converted to full URLs).
    user_prompt += "Links (some might be relative links):\n"
    # \n.join(website.links) converts the list of links (website.links) into a newline-separated string, making it easier to read for the LLM.
    user_prompt += "\n".join(website.links)
    # After constructing the user prompt with the necessary instructions and list of links, this line returns the complete prompt, ready to be sent to the LLM.
    return user_prompt

In [11]:
# This prints the generated user prompt for the Website object ed, which represents the webpage "https://edwarddonner.com".
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links. 
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/08/06/outsmart/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/
http

_**get_links(url)**_ is responsible for extracting relevant links from a webpage and leveraging GPT-4o-mini to identify which of those links should be included in a company brochure (e.g., About page, Careers page). The output is returned as a JSON object for easy processing. Let's break it down:

In [28]:
def get_links(url):
    website = Website(url) # Create an instance of the Website class (which was defined earlier) using the provided URL.
    # 'openai.chat.completions.create()' sends a request to OpenAI's GPT-4o-mini model to generate a completion based on the system and user prompts.
    completion = openai.chat.completions.create(
        # a. Specifying the Model:
        model = MODEL, # The model used for the request is specified by the constant MODEL ( 'gpt-4o-mini' ).
        # b. Providing the Messages:
        messages = [
            # System Message (link_system_prompt): defines the role of the model (deciding which links are relevant for a brochure).
            {"role" : "system", "content" : link_system_prompt},
            # User Message (get_links_user_prompt(website)):creates the user-specific prompt, dynamically generated using the links found on the given webpage.
            {"role" : "user", "content" : get_links_user_prompt(website)}
        ],
        # c. Specifying the Response Format:
        response_format = {"type" : "json_object"}  # This specifies that the LLM should return the response in JSON format
    )
    # d. Extracting the LLM Response:
    # The response from the LLM is stored in the choices array. choices[0] refers to the first (and usually only) completion returned by the model.
    # message.content contains the generated JSON output, which will include the relevant links extracted from the webpage.
    result = completion.choices[0].message.content
    # e. Returning the Parsed JSON Result:
    return json.loads(result)

In [13]:
get_links("https://anthropic.com")

{'links': [{'type': 'home page', 'url': 'https://anthropic.com'},
  {'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}]}
Landing page: 
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
New
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Get started now
Our Work
Announcements
Claude 3.5 Sonnet
Jun 2

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to bulid a short brochure of the company in markdown. \n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 chacaters
    return user_prompt

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role" : "system", "content" : system_prompt},
            {"role" : "user", "content" : get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Welcome to Anthropic: Where AI Meets Safety and Hilarity!

**Unravel the Mysteries of AI, One Chuckle at a Time!**

### Meet Claude – Your Friendly AI Companion
While other AIs might just leave you perplexed, Claude 3.5 Sonnet is here to help **you** sound smart (or at least, slightly less clueless). Whether it's writing emails or debugging code, Claude's got your back! Just don't ask him to make you a sandwich. He hasn’t mastered culinary arts... yet.

### Building the Future, Because We Can!
At Anthropic, we genuinely believe that AI can save the world! Partners include businesses, nonprofits, and civil society groups—basically anyone who needs a reliable (and hilarious) AI system. Because let’s face it, who wouldn’t want an AI buddy that makes productivity so easy, you might forget to shower?

### Company Culture: Where No One is Left Alone (Except During the Office **Plant** Care Rotation)
1. **Here for the Mission**: We’re like superheroes, but our capes are filled with data and love for safe AI.
2. **Unusually High Trust**: In a world of chaos, we assume goodwill and disagree without throwing coffee cups. And yes, we *really* do take our meetings seriously (after snack time).
3. **One Big Team**: Everyone here is treated like a key player. No sitting at the kiddie table—unless it's lunch, in which case, all bets are off.
4. **Keep It Simple**: Why find a complex solution when a rubber band and duct tape will do the trick?

### Careers at Anthropic: Join Us in Our Quest!
Looking for your next adventure? Our team of engineers, researchers, policy experts, and maybe a random space cat is constantly looking to grow. With unlimited PTO, generous parental leave, and daily catered lunches, your work-life balance will be impeccable (even your most ambitious desk plant will appreciate it)!

**Benefits Include:**
- Health, dental, and vision plans that won’t drain your savings account faster than Claude can generate a report.
- Unlimited PTO—because life happens! (We’ll leave the taking-kids-to-school thoughts to you).
- Options for equity donations—because sharing is caring! 

### Join Our Great Escape—To AI Paradise!
If you’re an aspiring AI aficionado or just someone whose hobbies include reading tech blogs and petting plants, check our **open roles**! You’ll go through an interview process that will help you discover if you’re a good fit and not just someone who “knows a guy.” 

### Claude: More than Just an AI
Our AI can help you brainstorm ideas, solve deep mathematical questions, and provide uplifting quotes from your favorite movies— like the Terminator. But remember, with great power comes great responsibility... to not make him bitter about gathering data when asked to make one more report!

### Ready to Take the Plunge?
Explore the possibilities at **Anthropic**! Partner with Claude (he won’t make you wear matching outfits) and let’s create a fun, safe, and productive AI-driven world together! 

---

Don’t just take our word for it, “Claude helped us generate insightful metrics reports instantly!” – *A Very Happy Customer* (**Disclaimer**: Not an actual quote; we just needed a line here). 

For more info, visit our website or send us an email! After all, communication is key—but if Claude ever learns sarcasm, we’re doomed. 

---

*Anthropic PBC: Where AI Safety and Humor Intertwine in Perfect Peace!* 🪴💻

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model = MODEL,
        messages = [
             {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream = True
    )

    response = ""
    display_handle = display(Markdown(""), display_id = True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("'''", "").replace("markdown","")
        update_display(Markdown(response), display_id = display_handle.display_id)

In [25]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Welcome to Anthropic: Where AI Gets a Safety Net! 🤖🪂

### Meet Claude: Your New AI Buddy! 🎤✨
Ever wished you had a personal AI assistant who could make your workload disappear? Introducing Claude 3.5 Sonnet! Claude doesn’t just help you write emails – he might even take over your whole job (joking... mostly)! With Claude, your productivity will skyrocket faster than a rocket coming out of NASA! 

### Who We Are 🧑‍🔬💼
Nestled in the sunny tech paradise of San Francisco, Anthropic is on a mission to build AI systems that are as reliable as your grandma's famous cookie recipe. We’re not just a team; we’re a lively blend of researchers, engineers, policy experts – and a few plant enthusiasts (we do love our greens). Our interdisciplinary crew knows how to party but also knows that safety isn’t just a buzzword; it’s our way of life!

### Company Culture: We Trust You! 🤝🎉
At Anthropic, we have an unusually high trust environment. Meaning if you see someone wearing mismatched socks, it’s likely an intentional fashion choice and not a sign of mental breakdown. We assume good faith, disagree kindly, and encourage emotional maturity (no throwing tantrums allowed!) Collaboration is our jam – every team member's voice is welcome, and we might even harmonize if we get carried away.

### What Makes Us Tick? 🕰️💡
1. **Here for the Mission**: We’re passionate about AI benefiting society – we can’t do it alone, though! 
2. **Unusually High Trust**: We thrive on honesty and kindness. 
3. **One Big Team**: Think of us like the Avengers, minus the capes (or maybe with really cool capes).
4. **Simple Solutions**: Sometimes the simplest solution is the best – like taking a nap instead of dealing with problems (just kidding, please don’t nap at work!).

### Our Customers: Who Loves Us? 🥰🚀
From startups to enterprises, we’re transforming how businesses operate with Claude. Customers include forward-thinking entities like Salesforce and IG Group who trust us to enhance their capabilities. It's like getting the Avengers, but for AI systems! 

### Join Us to Build the Future! 🌟👩‍💻
Looking for a job that feeds your passion as much as your belly (daily lunches, yum)? Explore opportunities across engineering, policy, and research. 
- **Benefits**: Generous parental leave, unlimited PTO, even a flexible wellness stipend, because we're really into happiness here!
- **Why Anthropic?**: We don’t require PhDs to join the party; about 50% of our Technical Staff skipped formal education. Creativity and ingenuity are the only tickets required!

### Our Research - Advancing with Safety 🔬🚧
We believe that AI has a colossal impact on the world. It has the potential to give you a laugh, help you out in a bind, or even suggest the perfect pizza topping. But with great power comes great responsibility! Our research teams are continuously developing and applying safety techniques to ensure AI is as harmless as a kitten (with a really good resume).

### Let Claude Help You Out! 🎉🎨
Whether you need help drafting emails, brainstorming the next big idea, or troubleshooting code, Claude's got your back! Just don’t ask him to take your coffee order – he’s not quite there yet. 

Ready to join us on this wild ride of AI innovation? Let’s create something safer, smarter, and a little more fun together! 🌍✨

---

**Connect with Us**
- 🌐 [Website](https://www.anthropic.com)
- 📧 [Press Inquiries](mailto:press@anthropic.com)
- 📱 Follow us on Social Media!

**Anthropic: Building AI You Can Trust. 🛠️**

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [27]:
stream_brochure("YahooFinance","https://finance.yahoo.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.yahoo.com/'}, {'type': 'about page', 'url': 'https://www.yahooinc.com/careers/'}, {'type': 'finance page', 'url': 'https://finance.yahoo.com/'}, {'type': 'jobs page', 'url': 'https://sports.yahoo.com/jobs/'}]}


# Welcome to Yahoo Finance: Where Money Talks (and Sometimes Cries)!

---

### About Us
At Yahoo Finance, we don’t just report on the stock market; we give it a pulpit! Whether it's stocks, bonds, or the latest crypto craze, we have your finances covered with a hearty helping of news and a sprinkle of humor. 

### What We Offer:
- **Breaking News**: When the stock market has a meltdown, we're the first to give you the scoop (and maybe a virtual hug).
- **Stock Market Live**: Tracking stock prices is like watching a season finale of your favorite show – you never know if it’ll end with joy or despair!
- **Investment Ideas**: Want to know which stocks are trending? We’ve got the lowdown, saving you from rubbernecking the Wall Street highways.
  
### Our Customers: 
From busy professionals to the financially curious, our audience is as diverse as our news content. Remember, whether you’re a savvy investor or a daydreamer hoping to retire on a sunny beach with a piña colada, there’s something for everyone! 

### Careers at Yahoo Finance:
**Why Join the Family?**
- **Culture**: We’re not just a team; we're a quirky family who believes 'Finance' is just a fancy way of saying “a never-ending adventure.” 
- **Work Perks**: From financial wellness programs (so you can afford those piña coladas!) to flexible working hours, we ensure that our employees are not just number crunchers but also number enthusiasts.
  
### Job Opportunities:
- **Finance Analysts**: Show off your hidden love for Excel spreadsheets!
- **Content Creators**: Bring your financial HOT takes to the world. Can you make 'boring' sound 'exciting'? We can! 
- **Marketing Masters**: Help people discover the treasure chest of financial wisdom we offer. 

---

### Get Ready to Roll!
If your money could talk, it would probably scream “Yahoo Finance!” So grab your calculators, a comfy seat, and prepare to dive into the exciting world of stocks, bonds, and everything in between. 

**Join us today!** Here’s the trade-off: your sanity may go down, but your finances could go up!

> *Disclaimer: We cannot guarantee that following us will result in wealth. Side effects may include sudden confidence in stock picking, spontaneous premium subscriptions, and an addiction to our daily newsletters.*

---

**Contact Us at:**  
📧 finance-fun@Yahoo.com  
🌐 [Visit Yahoo Finance](https://finance.yahoo.com) 

**Let’s talk stock!** 🤑